# Supporting code and data for "Automatic Bug Minimization in Proof Assistants: A Case Study in Coq"

In [1]:
%matplotlib inline

import os
import sys
print(f'Python {sys.version}')

import IPython
from IPython.core.display import display, HTML
print(f'IPython {IPython.__version__}')

print('\nLibraries:\n')

import csv
print(f'csv {csv.__version__}')

import matplotlib
import matplotlib.pyplot as plt
print(f'matplotlib {matplotlib.__version__}')

import numpy as np
print(f'numpy {np.__version__}')

import pandas as pd
from pandas.plotting import register_matplotlib_converters
print(f'pandas {pd.__version__}')

import re
print(f're {re.__version__}')

import requests
print(f'requests {requests.__version__}')

Python 3.9.6 (default, Jun 28 2021, 08:57:49) 
[GCC 10.3.0]
IPython 7.24.1

Libraries:

csv 1.0
matplotlib 3.4.2
numpy 1.20.3
pandas 1.2.4
re 2.2.1
requests 2.25.1


## Data collection

We use the GitHub GraphQL API because it allows fetching only the information we need, and at a much faster rate (we can get up to 100 nodes in a single request). Getting all the objects of a certain type requires then to repeat the request to go through all the pages of results.

You need to provide a personal `api_token` if you want to get fresh data from GitHub. Otherwise, this notebook will skip the data collection step and load the CSV files from the local filesystem.

In [2]:
api_token = ''

In [3]:
def requestAllPages(query,rows_and_next_variables,filename,columns):
  if api_token == '':
    return
  headers = {'Authorization': f'token {api_token}'}
  url = 'https://api.github.com/graphql'
  rows, variables = rows_and_next_variables(None)
  while len(variables)>0:
    json = {'query':query,'variables':variables.pop()}
    r = requests.post(url=url, json=json, headers=headers)
    if r.status_code == 403:
      print('Unauthorized request:')
      print(json)
    r.raise_for_status() # Abort if unsuccessful request
    new_rows, next_variables = rows_and_next_variables(r.json()['data'])
    rows += new_rows
    variables += next_variables
  if len(rows) > 0:
    with open(filename, 'w') as f:
      writer = csv.writer(f)
      writer.writerow(columns)
      writer.writerows(rows)

We search all PRs where CI minimization was proposed (excluding those authored by Jason Gross, which were mostly to debug the minimizer) and we retrieve all the comments from coqbot-app to know what happened. We only keep the first 15 lines of each comment, to reduce the size of the CSV file, because these lines will contain all the information we need.

Make sure to uncomment the last line and to provide an `api_token` to re-run this.

In [4]:
def fetch_pr_comments():

  query = """
    query commentQuery($number: Int!, $single: Boolean!, $prCursor: String, $commentCursor: String) {
      search(query: "repo:coq/coq coqbot ci minimize", type:ISSUE, first: 10, after: $prCursor) @skip (if: $single) {
        pageInfo {
          endCursor
          hasNextPage
        }
        nodes {
          ... pullRequest
        }
      }
      repository(owner: "coq", name: "coq") @include (if: $single) {
        pullRequest(number: $number) {
          ... pullRequest
        }
      }
    }

    fragment pullRequest on PullRequest {
      number
      author { login }
      comments(first: 50, after: $commentCursor) {
        pageInfo {
          endCursor
          hasNextPage
        }
        nodes {
          createdAt
          author { login }
          bodyText
          databaseId
        }
      }
    }
  """

  def treat_pr(pr):
    rows, variables = [], []
    number = pr['number']
    if pr['author']['login'] != 'JasonGross':
      for comment in pr['comments']['nodes']:
        if comment['author']['login'] == "coqbot-app":
          date = pd.to_datetime(comment['createdAt']).tz_localize(None)
          body = '\\n'.join(comment['bodyText'].split('\n')[:15])
          rows.append([comment['databaseId'],number,date,body])
    if pr['comments']['pageInfo']['hasNextPage']:
      variables += [{
          'single':True,
          'number':number,
          'commentCursor':pr['comments']['pageInfo']['endCursor']
      }]
    return rows, variables

  def rows_and_next_variables(data):
    if data is None:
      return [], [{'single':False,'number':0}]
    else:
      if 'search' in data:
        prs = data['search']
        rows, variables = [], []
        for pr in prs['nodes']:
          if 'number' in pr:
            new_rows, new_variables = treat_pr(pr)
            rows += new_rows
            variables += new_variables
        if prs['pageInfo']['hasNextPage']:
          variables += [{
              'single':False,
              'number':0,
              'prCursor':prs['pageInfo']['endCursor']
          }]
        return rows, variables
      else:
        return treat_pr(data['repository']['pullRequest'])

  requestAllPages(
      query,
      rows_and_next_variables,
      'pr_comments.csv',
      ['id','number','date','body']
  )

# fetch_pr_comments()

## Data processing

We retrieve the saved dataset from disk.

In [5]:
coqbot_comments = pd.read_csv('pr_comments.csv',parse_dates=['date'],index_col=0)

The data retrieved from GitHub can contain duplicated entries (same ID).

In [6]:
coqbot_comments[coqbot_comments.index.duplicated()]

,number,date,body
id,,,
858807919,14480,2021-06-10 17:20:46,"Hey, I have detected that there were CI failur..."
858810041,14480,2021-06-10 17:23:29,I am now running minimization at commit fc4887...
858903523,14480,2021-06-10 18:48:21,Minimized File /github/workspace/builds/coq/co...


We remove them:

In [7]:
coqbot_comments = coqbot_comments[~coqbot_comments.index.duplicated(keep='first')]

### Matching CI minimize comments

We look for comments marking the beginning and the end of the minimization. We only keep the first run for each pull request and minimized project to avoid double counting minimization examples.

In [8]:
coqbot_comments = coqbot_comments.assign(targets = coqbot_comments['body'].str.extract(r'I (?:have initiated|am now running) minimization at commit [a-z0-9]* (?:for the suggested targets? |on requested targets? )(?P<targets>[^.]+)'))
coqbot_comments_with_targets = coqbot_comments[~coqbot_comments['targets'].isna()]
coqbot_comments_with_targets

,number,date,body,targets
id,,,,
996111227,15274,2021-12-16 19:23:51,I have initiated minimization at commit ccd2e4...,ci-category_theory as requested
1001007062,15400,2021-12-25 11:36:55,I am now running minimization at commit 567431...,ci-color
957075875,14727,2021-11-02 03:47:08,I have initiated minimization at commit 9b18fa...,ci-fcsl_pcm as requested
1017935452,15518,2022-01-20 21:17:11,I have initiated minimization at commit 8b7024...,"ci-argosy, ci-bedrock2, ci-category_theory, ci..."
1018009024,15518,2022-01-20 23:13:25,I am now running minimization at commit 8b7024...,ci-category_theory
...,...,...,...,...
872606710,13107,2021-07-01 23:17:44,I am now running minimization at commit 7c1593...,ci-tlc
872607702,13107,2021-07-01 23:20:22,I am now running minimization at commit 7c1593...,ci-bedrock2
872629235,13107,2021-07-02 00:26:29,I am now running minimization at commit 7c1593...,"ci-bedrock2, ci-tlc"


In [9]:
targets = coqbot_comments_with_targets['targets'].str.extractall(r'(?P<target>ci-[^,\s]*)')
# Exclude accidental runs on the Coq bug minimizer itself
targets = targets[targets['target'] != 'ci-coq_tools']
# ci-metacoq build system was responsible for many failures, if we exclude it we boost our success score from 75 to 79%
# targets = targets[targets['target'] != 'ci-metacoq']
minimization_started_comments = targets.join(coqbot_comments_with_targets).sort_values('date').drop_duplicates(subset=['target','number']).set_index(['target','number'])[['date']]
minimization_started_comments

,,date
target,number,
ci-mathcomp,13969,2021-05-21 22:34:08
ci-equations,13969,2021-05-26 14:10:29
ci-fourcolor,13969,2021-05-26 14:10:29
ci-iris,13969,2021-05-26 14:10:29
ci-perennial,13969,2021-05-26 14:10:29
...,...,...
ci-relation_algebra,15518,2022-01-20 21:17:11
ci-unimath,15518,2022-01-20 21:17:11
ci-verdi_raft,15518,2022-01-20 21:17:11


For successful minimization runs (i.e., runs that produced a minimized file, and were not automatically restarted after being interrupted by a timeout), we extract information from the headers of the minimized files, such as the expected `coqc` runtime on this file, the initial number of lines, and the final number of lines, or if any module couldn't be inlined.

In [10]:
minimization_success_comments = coqbot_comments[coqbot_comments['body'].str.startswith('Minimized File') & ~coqbot_comments['body'].str.contains('interrupted by timeout, being automatically continued')]
minimization_success_comments = minimization_success_comments.assign(
    runtime = minimization_success_comments['body'].str.extract(r'Expected coqc runtime on this file: ([0-9\.]+) sec').astype(float),
    initial_size = minimization_success_comments['body'].str.extract(r'from original input, then from ([0-9]+) lines').astype(float),
    final_size = minimization_success_comments['body'].str.extract(r'to ([0-9]+) lines \*\)').astype(float),
    inline_failure = minimization_success_comments['body'].str.contains('could not be inlined'),
    truncated = minimization_success_comments['body'].str.contains('Minimized Coq File \(truncated')
)
minimization_success_comments

,number,date,body,targets,runtime,initial_size,final_size,inline_failure,truncated
id,,,,,,,,,
996116369,15274,2021-12-16 19:31:28,Minimized File /github/workspace/builds/coq/co...,NaN,0.213,321.0,124.0,False,False
1001010494,15400,2021-12-25 12:16:25,Minimized File /github/workspace/builds/coq/co...,NaN,1.018,522.0,81.0,True,False
957082119,14727,2021-11-02 04:02:44,Minimized File /github/workspace/builds/coq/co...,NaN,0.159,613.0,17.0,False,False
1017939088,15518,2022-01-20 21:22:16,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True
1017939694,15518,2022-01-20 21:23:10,Minimized File /github/workspace/builds/coq/co...,NaN,0.117,442.0,31.0,False,False
...,...,...,...,...,...,...,...,...,...
872648442,13107,2021-07-02 01:28:51,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,2300.0,128.0,False,False
872665027,13107,2021-07-02 02:22:08,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,1465.0,133.0,False,False
873330611,13107,2021-07-03 02:28:22,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,409.0,21.0,False,False


There are a few files for which we couldn't retrieve any info from the headers (or with a header with absurd data). These actually correspond to files where the minimizer was not able to reproduce any error message and thus did not really produce a reduced case. Thus, we should count this together with the minimization failures:

In [11]:
minimization_false_successes = minimization_success_comments[minimization_success_comments['initial_size'].isna() | (minimization_success_comments['initial_size'] == 0)]
minimization_success_comments = minimization_success_comments[~minimization_success_comments['initial_size'].isna() & (minimization_success_comments['initial_size'] > 0)]
minimization_false_successes

,number,date,body,targets,runtime,initial_size,final_size,inline_failure,truncated
id,,,,,,,,,
1017939088,15518,2022-01-20 21:22:16,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True
1013700274,15487,2022-01-15 15:26:47,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True
901281069,14746,2021-08-18 17:06:24,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False
960971850,14746,2021-11-04 13:49:54,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False
961175514,14746,2021-11-04 15:50:38,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False
961477317,14746,2021-11-04 22:21:02,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False
886230665,11966,2021-07-25 17:04:47,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False
962299051,15128,2021-11-06 01:05:52,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True
965630699,15128,2021-11-10 18:38:43,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True


For these reduced test cases, we also compute the total number of removed lines by aggregating the information found in the headers about the number of lines removed at each step.

In [12]:
line_reduction = minimization_success_comments['body'].str.extractall(r'then from (?P<reduced_from>[0-9]+) lines to (?P<reduced_to>[0-9]+) lines')
line_reduction = line_reduction.assign(total_line_reduction = line_reduction['reduced_from'].astype(int) - line_reduction['reduced_to'].astype(int))
line_reduction = line_reduction.groupby(level=[0]).sum()
minimization_success_comments = minimization_success_comments.join(line_reduction['total_line_reduction'])
minimization_success_comments

,number,date,body,targets,runtime,initial_size,final_size,inline_failure,truncated,total_line_reduction
id,,,,,,,,,,
996116369,15274,2021-12-16 19:31:28,Minimized File /github/workspace/builds/coq/co...,NaN,0.213,321.0,124.0,False,False,256
1001010494,15400,2021-12-25 12:16:25,Minimized File /github/workspace/builds/coq/co...,NaN,1.018,522.0,81.0,True,False,465
957082119,14727,2021-11-02 04:02:44,Minimized File /github/workspace/builds/coq/co...,NaN,0.159,613.0,17.0,False,False,650
1017939694,15518,2022-01-20 21:23:10,Minimized File /github/workspace/builds/coq/co...,NaN,0.117,442.0,31.0,False,False,416
1017939985,15518,2022-01-20 21:23:38,Minimized File /github/workspace/builds/coq/co...,NaN,0.134,76.0,25.0,False,False,56
...,...,...,...,...,...,...,...,...,...,...
872648442,13107,2021-07-02 01:28:51,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,2300.0,128.0,False,False,2160
872665027,13107,2021-07-02 02:22:08,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,1465.0,133.0,False,False,1417
873330611,13107,2021-07-03 02:28:22,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,409.0,21.0,False,False,410


In [13]:
minimization_failure_comments = coqbot_comments[coqbot_comments['body'].str.startswith('Error: Could not minimize file')].append(minimization_false_successes)
minimization_failure_comments

,number,date,body,targets,runtime,initial_size,final_size,inline_failure,truncated
id,,,,,,,,,
1017937526,15518,2022-01-20 21:20:13,Error: Could not minimize file (from ci-sf) (...,NaN,NaN,NaN,NaN,NaN,NaN
1017937646,15518,2022-01-20 21:20:22,Error: Could not minimize file (from ci-itaut...,NaN,NaN,NaN,NaN,NaN,NaN
1017937663,15518,2022-01-20 21:20:24,Error: Could not minimize file (from ci-relat...,NaN,NaN,NaN,NaN,NaN,NaN
1017937985,15518,2022-01-20 21:20:54,Error: Could not minimize file (from ci-metac...,NaN,NaN,NaN,NaN,NaN,NaN
1017944365,15518,2022-01-20 21:29:39,Error: Could not minimize file (from ci-categ...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
881982360,14671,2021-07-18 01:22:03,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False
849735330,13895,2021-05-27 15:37:09,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True
849176004,14392,2021-05-26 23:04:20,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True


We aggregate successful and failed end minimization comments and only keep the first run for each project and pull request:

In [14]:
minimization_finished_comments = minimization_success_comments.assign(success=True).append(minimization_failure_comments.assign(success=False))
targets = minimization_finished_comments['body'].str.extract(r'(?P<target>ci-[^)]*)')
# Exclude accidental runs on the Coq bug minimizer itself
targets = targets[targets['target'] != 'ci-coq_tools']
minimization_finished_comments = targets.join(minimization_finished_comments).sort_values('date').drop_duplicates(subset=['target','number']).set_index(['target','number'])#[['date','success','runtime']]
minimization_finished_comments

,,date,body,targets,runtime,initial_size,final_size,inline_failure,truncated,total_line_reduction,success
target,number,,,,,,,,,,
ci-mathcomp,13969,2021-05-21 22:37:15,Error: Could not minimize file (from ci-mathc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
ci-iris,13969,2021-05-26 14:13:55,Error: Could not minimize file (from ci-iris)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
ci-equations,13969,2021-05-26 14:14:19,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True,NaN,False
ci-fourcolor,13969,2021-05-26 14:15:27,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False,NaN,False
ci-perennial,13969,2021-05-26 14:16:08,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,1726.0,3.0,False,False,3455.0,True
...,...,...,...,...,...,...,...,...,...,...,...
ci-iris,15518,2022-01-20 21:48:56,Minimized File /github/workspace/builds/coq/co...,NaN,0.265,457.0,270.0,False,False,258.0,True
ci-compcert,15518,2022-01-20 21:53:07,Minimized File /github/workspace/builds/coq/co...,NaN,0.339,4967.0,51.0,False,False,4957.0,True
ci-coqprime,15518,2022-01-20 22:25:52,Minimized File /github/workspace/builds/coq/co...,NaN,0.488,464.0,474.0,False,False,110.0,True


We match these with the comments corresponding to the beginning of the minimization to compute the minimization duration:

In [15]:
minimization_pairs = minimization_started_comments.join(minimization_finished_comments,lsuffix='_start',rsuffix='_end')
minimization_pairs = minimization_pairs.assign(duration=(minimization_pairs['date_end'] - minimization_pairs['date_start']).dt.seconds)
minimization_pairs

,,date_start,date_end,body,targets,runtime,initial_size,final_size,inline_failure,truncated,total_line_reduction,success,duration
target,number,,,,,,,,,,,,
ci-mathcomp,13969,2021-05-21 22:34:08,2021-05-21 22:37:15,Error: Could not minimize file (from ci-mathc...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,187.0
ci-equations,13969,2021-05-26 14:10:29,2021-05-26 14:14:19,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True,NaN,False,230.0
ci-fourcolor,13969,2021-05-26 14:10:29,2021-05-26 14:15:27,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False,NaN,False,298.0
ci-iris,13969,2021-05-26 14:10:29,2021-05-26 14:13:55,Error: Could not minimize file (from ci-iris)...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,206.0
ci-perennial,13969,2021-05-26 14:10:29,2021-05-26 14:16:08,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,1726.0,3.0,False,False,3455.0,True,339.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ci-relation_algebra,15518,2022-01-20 21:17:11,2022-01-20 21:20:24,Error: Could not minimize file (from ci-relat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,193.0
ci-unimath,15518,2022-01-20 21:17:11,2022-01-20 21:26:33,Minimized File /github/workspace/builds/coq/co...,NaN,0.150,3947.0,79.0,False,False,3908.0,True,562.0
ci-verdi_raft,15518,2022-01-20 21:17:11,2022-01-20 21:27:38,Minimized File /github/workspace/builds/coq/co...,NaN,0.172,618.0,53.0,False,False,588.0,True,627.0


For the successful cases, there are other data that we can analyze:

In [16]:
successful_minimization_pairs = minimization_pairs[minimization_pairs['success'] == True][['date_start','duration','runtime','initial_size','final_size','inline_failure','truncated','total_line_reduction']]
successful_minimization_pairs

,,date_start,duration,runtime,initial_size,final_size,inline_failure,truncated,total_line_reduction
target,number,,,,,,,,
ci-perennial,13969,2021-05-26 14:10:29,339.0,NaN,1726.0,3.0,False,False,3455.0
ci-quickchick,13969,2021-05-26 14:10:29,534.0,NaN,1119.0,66.0,False,False,1074.0
ci-hott,13126,2021-06-07 14:13:54,8548.0,NaN,294.0,483.0,False,False,1669.0
ci-quickchick,13072,2021-06-09 15:18:55,1399.0,NaN,121.0,150.0,False,False,377.0
ci-rewriter,13072,2021-06-09 15:18:55,1228.0,NaN,2114.0,219.0,False,False,1983.0
...,...,...,...,...,...,...,...,...,...
ci-iris,15518,2022-01-20 21:17:11,1905.0,0.265,457.0,270.0,False,False,258.0
ci-rewriter,15518,2022-01-20 21:17:11,359.0,0.117,442.0,31.0,False,False,416.0
ci-unimath,15518,2022-01-20 21:17:11,562.0,0.150,3947.0,79.0,False,False,3908.0


Number of pull requests these minimization pairs come from:

In [17]:
len(minimization_pairs.index.to_frame()[['number']].drop_duplicates())

51

### Results

#### RQ1: How often does the minimizer produce a reduced test case?

Proportion of the time the minimizer successfully produced a reduced file:

In [18]:
len(successful_minimization_pairs)/len(minimization_pairs)

0.7539267015706806

Let's take a look at the failed runs. First, the runs for which no finish comment was found:

In [19]:
minimization_pairs[minimization_pairs['date_end'].isna()]

date_start date_end body targets  \
target                number                                             
ci-vst                14746  2021-08-06 14:10:50      NaT  NaN     NaN   
                      14777  2021-08-13 12:21:08      NaT  NaN     NaN   
ci-metacoq            14819  2021-09-02 13:49:41      NaT  NaN     NaN   
ci-bedrock2           15128  2021-11-06 00:58:32      NaT  NaN     NaN   
ci-fiat_crypto_legacy 15178  2021-11-12 17:27:33      NaT  NaN     NaN   

                              runtime  initial_size  final_size  \
target                number                                      
ci-vst                14746       NaN           NaN         NaN   
                      14777       NaN           NaN         NaN   
ci-metacoq            14819       NaN           NaN         NaN   
ci-bedrock2           15128       NaN           NaN         NaN   
ci-fiat_crypto_legacy 15178       NaN           NaN         NaN   

                             inline_failure truncated  total_line_reduction  \
target                number                                                  
ci-vst                14746             NaN       NaN                   NaN   
                      14777             NaN       NaN                   NaN   
ci-metacoq            14819             NaN       NaN                   NaN   
ci-bedrock2           15128             NaN       NaN                   NaN   
ci-fiat_crypto_legacy 15178             NaN       NaN                   NaN   

                             success  duration  
target                number                    
ci-vst                14746      NaN       NaN  
                      14777      NaN       NaN  
ci-metacoq            14819      NaN       NaN  
ci-bedrock2           15128      NaN       NaN  
ci-fiat_crypto_legacy 15178      NaN       NaN

In the last four cases, there was an infinite loop and the minimizer had to be stopped manually.

Next, the runs for which minimization concluded with an explicit error:

In [20]:
minimization_pairs[(minimization_pairs['success'] == False) & minimization_pairs['body'].str.startswith('Error: Could not minimize file')]

date_start            date_end  \
target                   number                                           
ci-mathcomp              13969  2021-05-21 22:34:08 2021-05-21 22:37:15   
ci-iris                  13969  2021-05-26 14:10:29 2021-05-26 14:13:55   
ci-perennial             14392  2021-05-26 20:55:06 2021-05-26 20:58:29   
ci-coq_performance_tests 12743  2021-05-31 21:18:34 2021-05-31 21:33:24   
ci-metacoq               14234  2021-06-02 00:32:18 2021-06-02 00:43:52   
                         13072  2021-06-09 15:18:55 2021-06-09 15:30:20   
ci-verdi_raft            13072  2021-06-09 15:53:03 2021-06-09 15:57:55   
ci-metacoq               14137  2021-06-14 09:42:50 2021-06-14 09:45:53   
                         14253  2021-06-21 19:07:57 2021-06-21 19:10:47   
ci-elpi                  13107  2021-06-29 14:41:10 2021-06-29 14:44:05   
                         14798  2021-08-21 15:35:53 2021-08-21 15:40:04   
ci-hott                  14929  2021-10-22 15:26:26 2021-10-22 15:29:38   
ci-metacoq               15128  2021-11-06 00:58:32 2021-11-06 01:02:56   
                         15390  2021-12-21 10:31:15 2021-12-21 10:35:27   
                         15518  2022-01-20 21:17:11 2022-01-20 21:20:54   
ci-category_theory       15518  2022-01-20 21:17:11 2022-01-20 21:29:39   
ci-itauto                15518  2022-01-20 21:17:11 2022-01-20 21:20:22   
ci-sf                    15518  2022-01-20 21:17:11 2022-01-20 21:20:13   
ci-relation_algebra      15518  2022-01-20 21:17:11 2022-01-20 21:20:24   

                                                                              body  \
target                   number                                                      
ci-mathcomp              13969   Error: Could not minimize file  (from ci-mathc...   
ci-iris                  13969   Error: Could not minimize file  (from ci-iris)...   
ci-perennial             14392   Error: Could not minimize file  (from ci-peren...   
ci-coq_performance_tests 12743   Error: Could not minimize file /github/workspa...   
ci-metacoq               14234   Error: Could not minimize file  (from ci-metac...   
                         13072   Error: Could not minimize file  (from ci-metac...   
ci-verdi_raft            13072   Error: Could not minimize file /github/workspa...   
ci-metacoq               14137   Error: Could not minimize file  (from ci-metac...   
                         14253   Error: Could not minimize file  (from ci-metac...   
ci-elpi                  13107   Error: Could not minimize file  (from ci-elpi)...   
                         14798   Error: Could not minimize file  (from ci-elpi)...   
ci-hott                  14929   Error: Could not minimize file  (from ci-hott)...   
ci-metacoq               15128   Error: Could not minimize file  (from ci-metac...   
                         15390   Error: Could not minimize file  (from ci-metac...   
                         15518   Error: Could not minimize file  (from ci-metac...   
ci-category_theory       15518   Error: Could not minimize file  (from ci-categ...   
ci-itauto                15518   Error: Could not minimize file  (from ci-itaut...   
ci-sf                    15518   Error: Could not minimize file  (from ci-sf) (...   
ci-relation_algebra      15518   Error: Could not minimize file  (from ci-relat...   

                                targets  runtime  initial_size  final_size  \
target                   number                                              
ci-mathcomp              13969      NaN      NaN           NaN         NaN   
ci-iris                  13969      NaN      NaN           NaN         NaN   
ci-perennial             14392      NaN      NaN           NaN         NaN   
ci-coq_performance_tests 12743      NaN      NaN           NaN         NaN   
ci-metacoq               14234      NaN      NaN           NaN         NaN   
                         13072      NaN      NaN           NaN         NaN   
ci-verdi_raft            13072    

Some comments were recorded on these errors in the tracking issue https://github.com/coq-community/run-coq-bug-minimizer/issues/8 or directly in the PR discussion.

|target | number | comment |
|-------|--------|---------|
| ci-mathcomp | 13969 | download issue? |
| ci-iris | 13969 | coqchk anomaly |
| ci-perennial | 14392 | base check not finished |
| ci-coq_performance_tests | 12743 | nondeterministic bug in Coq's native compilation handling, now a workaround is implemented |
| ci-metacoq | 14234 | minimization of metacoq did not work yet because of https://github.com/coq/coq/issues/14453 |
| ci-metacoq | 13072 | build system idempotency issue: ci-local target is not idempotent https://github.com/MetaCoq/metacoq/issues/563 |
| ci-verdi_raft | 13072 | issue with ambiguous libname prefixes https://github.com/JasonGross/coq-tools/issues/66 |
| ci-metacoq | 14137 | last case where the metacoq minimization did not work |
| ci-metacoq | 14253 | CI artifacts had expired |
| ci-elpi | 13107 | output test-suite fails with wrapper Coq binaries because file paths are absolutized https://github.com/math-comp/hierarchy-builder/issues/256 |
| ci-elpi | 14798 | same issue |
| ci-hott | 14929 | CI artifact download issue |
| ci-metacoq | 15128 | https://github.com/MetaCoq/metacoq/issues/605 |
| ci-metacoq | 15390 | same issue |
| ci-metacoq, ci-category_theory, ci-itauto, ci-sf, ci-relation_algebra | 15518 | the corresponding base jobs were skipped |

Finally, the runs which produced a minimized file but, from the header comments or their absence thereof, we can conclude that the minimization process failed to start (e.g., because it failed to reproduce the error message):

In [21]:
minimization_pairs[(minimization_pairs['success'] == False) & ~minimization_pairs['date_end'].isna() & minimization_pairs['body'].str.startswith('Minimized File')]

,,date_start,date_end,body,targets,runtime,initial_size,final_size,inline_failure,truncated,total_line_reduction,success,duration
target,number,,,,,,,,,,,,
ci-equations,13969,2021-05-26 14:10:29,2021-05-26 14:14:19,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True,NaN,False,230.0
ci-fourcolor,13969,2021-05-26 14:10:29,2021-05-26 14:15:27,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False,NaN,False,298.0
ci-interval,13895,2021-05-27 15:31:16,2021-05-27 15:37:09,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True,NaN,False,353.0
ci-hott,13269,2021-06-03 01:20:24,2021-06-03 01:24:50,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False,NaN,False,266.0
ci-fiat_parsers,13072,2021-06-09 15:18:55,2021-06-09 15:21:58,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True,NaN,False,183.0
ci-hott,14492,2021-06-12 21:33:20,2021-06-12 21:38:57,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False,NaN,False,337.0
ci-color,12493,2021-06-19 19:57:55,2021-06-19 20:03:50,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False,NaN,False,355.0
ci-equations,13107,2021-06-29 14:41:10,2021-06-29 14:44:15,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,False,NaN,False,185.0
ci-cross_crypto,13107,2021-06-29 14:41:10,2021-06-29 14:43:55,Minimized File /github/workspace/builds/coq/co...,NaN,NaN,NaN,NaN,False,True,NaN,False,165.0


Some comments were recorded on these errors in the tracking issue https://github.com/coq-community/run-coq-bug-minimizer/issues/8 or directly in the PR discussion.

|target | number | comment |
|-------|--------|---------|
| ci-equations | 13969 | mismatch between character locations and byte locations |
| ci-fourcolor | 13969 | `\r\n` line endings on Linux |
| ci-interval | 13895 | issue processing the error log (fixed) |
| ci-hott | 13269 | missing base artifacts |
| ci-fiat_parsers | 13072 | fixed in https://github.com/coq-community/run-coq-bug-minimizer/commit/38d5b1aedfc8944191276fc7be76b7c10a5e9fd9 |
| ci-hott | 14492 | no recognizable error? |
| ci-color | 12493 | failure because proxy constant names introduced by the PR change just from moving the Requires around |
| ci-equations | 13107 | error message parsing was too specific (fixed) |
| ci-cross_crypto | 13107 | namespace management issue (fixed) |
| ci-bedrock2 | 13107 | namespace management issue (fixed) |
| ci-coq_performance_tests | 13107 | issue using opam (fixed) |
| ci-tlc | 13107 | namespace management issue (fixed) |
| ci-perennial | 13107 | failure of run-minimizer to strip off -o arguments to coqc |
| ci-relation_algebra | 13107 | syntax error after stripping comments due a notation for ^* plus (x^*) showing up (fixed) |
| ci-fiat_crypto_legacy | 14671 | https://github.com/JasonGross/coq-tools/issues/107 |
| ci-metacoq | 11966 | https://github.com/JasonGross/coq-tools/issues/108 |
| ci-vst | 14783 | minimizer was misused by the user |
| ci-category_theory | 14818 | the error was independent from the PR |
| ci-category_theory | 12512 | Equations failed to install? |
| ci-cross_crypto | 14736 | probable failure of passing coqc due to eliminating Require after the buggy line |
| ci-fiat_crypto | 15128 | issue was that coqc ignores -top (fixed in the minimizer) |
| ci-bedrock2 | 15487 | Load results in miscomputed / misused glob file reported as https://github.com/coq/coq/issues/15497, fixed in the minimizer |
| ci-hott | 15518 | issue making universe names match in the error messages? |

#### RQ2: How often is this reduced test case fully standalone?

When are the reduced test cases fully standalone? We use the information from the header comments recording whether there was an inlining failure. However, this information was only added on 2021-10-21 (https://github.com/JasonGross/coq-tools/commit/d7217383349638adfba7c3a291a56237c37d84fc), so we only compute this for later runs.

In [22]:
runs_with_inlining_info = successful_minimization_pairs[successful_minimization_pairs['date_start'] >= pd.to_datetime('2021-10-21')]
no_inlining_failure = runs_with_inlining_info[runs_with_inlining_info['inline_failure'] == False]
len(runs_with_inlining_info)

47

In [23]:
len(no_inlining_failure) / len(runs_with_inlining_info)

0.8936170212765957

Let's take a look at the cases that were recorded with an inline failure:

In [24]:
runs_with_inlining_info[runs_with_inlining_info['inline_failure']==True]

date_start  duration  runtime  initial_size  \
target    number                                                        
ci-vst    15171  2021-11-12 11:13:07   19219.0   10.944         330.0   
ci-color  15400  2021-12-25 11:36:55    2370.0    1.018         522.0   
          15501  2022-01-18 13:13:16   19204.0    1.399         508.0   
ci-gappa  15501  2022-01-18 13:13:16     902.0    0.484         727.0   
ci-geocoq 15518  2022-01-20 21:17:11   16415.0    1.084        2242.0   

                  final_size inline_failure truncated  total_line_reduction  
target    number                                                             
ci-vst    15171        613.0           True     False                -234.0  
ci-color  15400         81.0           True     False                 465.0  
          15501        610.0           True     False                 181.0  
ci-gappa  15501        105.0           True     False                 664.0  
ci-geocoq 15518        201.0           True     False                2177.0

These failures to inline were diagnosed as such (and fixed):

|target | number | comment |
|-------|--------|---------|
| ci-vst | 15171 | the first inlining method failed and due to https://github.com/JasonGross/coq-tools/issues/110 (fixed), the other methods failed as well |
| ci-color | 15400 | robustness issue with error message https://github.com/JasonGross/coq-tools/issues/111 (fixed) |
| ci-color | 15501 | like ci-vst above, https://github.com/JasonGross/coq-tools/issues/110 |
| ci-gappa | 15501 | flocq not installing .glob files, fixed by https://github.com/coq/coq/pull/15509 |
| ci-geocoq | 15518 | like ci-vst above, https://github.com/JasonGross/coq-tools/issues/110 |

#### RQ3: How long does it take to produce such reduced test cases?

Duration for failed runs:

In [25]:
minimization_pairs[minimization_pairs['success'] == False]['duration'].describe(percentiles=[0.5,0.6,0.7,0.8,0.9,0.95])

count     42.000000
mean     306.095238
std      166.148745
min      165.000000
50%      258.000000
60%      295.600000
70%      314.800000
80%      354.600000
90%      442.700000
95%      693.550000
max      890.000000
Name: duration, dtype: float64

Duration for successful runs:

In [26]:
successful_minimization_pairs['duration'].describe(percentiles=[0.5,0.6,0.7,0.8,0.9])

count      144.000000
mean      6238.326389
std      11694.359851
min        232.000000
50%       1218.500000
60%       2199.800000
70%       4125.700000
80%       8396.400000
90%      19220.400000
max      73072.000000
Name: duration, dtype: float64

#### RQ4: What is the size of the reduced cases?

In [27]:
no_inlining_failure['final_size'].describe()

count      42.000000
mean      270.142857
std       459.723701
min        17.000000
25%        39.000000
50%       113.500000
75%       261.750000
max      2648.000000
Name: final_size, dtype: float64

In [28]:
successful_minimization_pairs['final_size'].describe()

count     144.000000
mean      367.784722
std       612.679569
min         3.000000
25%        35.000000
50%       123.000000
75%       466.500000
max      3804.000000
Name: final_size, dtype: float64

#### RQ5: How long do the reduced cases take to run?

For how many reduced cases do we have the expected runtime? What results do we get?

In [29]:
no_inlining_failure['runtime'].dropna().describe()

count    42.000000
mean      1.243929
std       4.446665
min       0.095000
25%       0.169500
50%       0.257500
75%       0.435500
max      26.506000
Name: runtime, dtype: float64

#### RQ6: What is the amount of code reduction?

How much smaller the final size is compared to the initial size?

In [30]:
np.mean(no_inlining_failure['final_size']/no_inlining_failure['initial_size'])

0.49821274790943026

In some cases, the final size can be larger than the initial size because of the dependency inlining process:

In [31]:
size_diffs = no_inlining_failure['initial_size']-no_inlining_failure['final_size']
len(size_diffs[size_diffs < 0])

6

In [32]:
len(size_diffs[size_diffs > 0])

36

Separate means for cases which were reduced and cases which were expanded:

In [33]:
np.mean(no_inlining_failure[size_diffs > 0]['final_size']/no_inlining_failure[size_diffs > 0]['initial_size'])

0.22097210947527876

In [34]:
np.mean(no_inlining_failure[size_diffs < 0]['final_size']/no_inlining_failure[size_diffs < 0]['initial_size'])

2.161656578514339

And when including all the dependencies that had to be inlined during the process?

In [35]:
np.mean(no_inlining_failure['final_size']/(no_inlining_failure['final_size'] + no_inlining_failure['total_line_reduction']))

0.31154996314933453

#### Project by project

This reveals significant differences but we have too little data to make any real conclusion:

In [36]:
successful_minimization_pairs.groupby(['target']).agg(['median','count'])

duration       runtime       initial_size        \
                        median count  median count       median count   
target                                                                  
ci-aac_tactics           232.0     1     NaN     0        279.0     1   
ci-argosy               3163.5     4  0.7140     1        540.0     4   
ci-bbv                   619.0     3  0.0950     1       7591.0     3   
ci-bedrock2             3322.0     9  0.1325     4        533.0     9   
ci-bignums              8529.0     1     NaN     0       1747.0     1   
ci-category_theory       634.0     3  0.2365     2        262.0     3   
ci-color                2758.0     5  1.2085     2        714.0     5   
ci-compcert             1197.0     5  0.3390     1        728.0     5   
ci-coqprime             4121.0     1  0.4880     1        464.0     1   
ci-coquelicot            461.5     2  0.6580     1       3775.0     2   
ci-cross_crypto         5953.0     1     NaN     0        934.0     1   
ci-equations             330.5     2     NaN     0        248.5     2   
ci-fcsl_pcm              936.0     1  0.1590     1        613.0     1   
ci-fiat_crypto          9525.0     4  0.1510     1        242.0     4   
ci-fiat_crypto_legacy    791.5    10  0.2250     3        836.5    10   
ci-fiat_parsers          730.5    10  0.3310     1        645.0    10   
ci-flocq                 834.0     1  0.4580     1        703.0     1   
ci-gappa                 902.0     1  0.4840     1        727.0     1   
ci-geocoq              10291.5     2  1.0840     1       1230.0     2   
ci-hott                 1857.0    13  0.4500     3        565.0    13   
ci-iris                 3397.0     8  0.2650     5        461.0     8   
ci-itauto               2936.0     1  0.3460     1      10995.0     1   
ci-math_classes         1076.5     4  0.1690     1        221.0     4   
ci-mathcomp            13729.0     4  1.0570     2       3520.0     4   
ci-menhir               1709.0     2  1.1340     1        454.0     2   
ci-metacoq               370.0     1     NaN     0       1124.0     1   
ci-mtac2                 326.5     2     NaN     0        251.0     2   
ci-perennial            2672.0     8  0.2465     4        601.5     8   
ci-quickchick            507.5     4  0.1300     1        179.5     4   
ci-relation_algebra      707.0     2     NaN     0        223.5     2   
ci-rewriter              910.0     9  0.2495     2       1339.0     9   
ci-sf                    736.0     1     NaN     0       2288.0     1   
ci-stdlib2               373.0     1     NaN     0        138.0     1   
ci-tlc                  4389.0     1     NaN     0        788.0     1   
ci-unimath              1323.5     4  0.1500     1       2435.0     4   
ci-verdi_raft            777.0     5  0.1825     2        618.0     5   
ci-vst                 15140.5     8  0.9890     3        872.0     8   

                      final_size       total_line_reduction        
                          median count               median count  
target                                                             
ci-aac_tactics              27.0     1                256.0     1  
ci-argosy                  369.0     4                467.0     4  
ci-bbv                      33.0     3               7547.0     3  
ci-bedrock2                464.0     9                293.0     9  
ci-bignums                1719.0     1                335.0     1  
ci-category_theory         105.0     3                256.0     3  
ci-color                   366.0     5                745.0     5  
ci-compcert                362.0     5                 94.0     5  
ci-coqprime                474.0     1                110.0     1  
ci-coquelicot               37.0     2               3742.5     2  
ci-cross_crypto            725.0     1                627.0     1  
ci-equations                31.5     2                221.0     2  
ci-fcsl_pcm                 17.0     1                650.0     1  
ci-fiat_crypt